# What Is This Notebook?

This notebook an exploration of using [zadd](https://redis.io/commands/zadd) add data into redis without having any duplicates. Instead of querying for data directly to determine if a piece of data has already been added into the database, we'll instead use the zadd command. We'll serialize information in json format then save it at a given timestamp. The save format will look a little bit like the following:

```py
{
    b"{...relevant data goes here}": float(timestamp) 
}
```

We'll be able to query all timeseries data using that timestamp we set inside of the zadd function. Technically it's called the `score` inside of redis. It's how we order separate keys of information.


Generally we'll be querying using the following concepts:

* [ZRANGEBYSCORE](https://redis.io/commands/zrangebyscore) - We'll be able to query by timeindex.
* [ZREMRANGEBYSCORE](https://redis.io/commands/zremrangebyscore) - We'll be able to remove data by time index. We could use it to remove the first, last, or any section of data if need be.
* [ZRANGE](https://redis.io/commands/zrange) - We'll be able to possibly get the last inputed item.



Generally the data is added into redis the following way:


```py

redis = Redis()

data = {"...key": float(timestamp), "...key2": float(timestamp)}

redis.zadd("event_key", *data)
```

In [2]:
%pwd

'/home/kevin/Code/OpenSource/fast_event_driven/test/notebooks'

In [3]:
%cd ../..

/home/kevin/Code/OpenSource/fast_event_driven


In [4]:
import uuid
import maya
import orjson
from copy import copy
from redis import Redis
from typing import List, Set
from jamboree.utils.helper import Helpers

In [5]:
import pandas as pd

In [6]:
import pandas_datareader.data as web
from pandas import Series, DataFrame, Timestamp

In [7]:
redis = Redis()
helper = Helpers()

# Helper Functions

---
Functions we need to officially handle the dictionary/set manipulation. They're heavily used inside of the save and query latest function.


### Using Maya to handle `epoch` time

`maya` is a heavily simplified time handling library. You might be wondering why we're using it over datetime and time. Well, that's because of all of the amazing features it has compared to the two native libraries. It allows us to have a variance of handling timeseries information.


Just have a look at the ways people can use it simply:


```py
>>> now = maya.now()
<MayaDT epoch=1481850660.9>

>>> tomorrow = maya.when('tomorrow')
<MayaDT epoch=1481919067.23>

>>> tomorrow.slang_date()
'tomorrow'

>>> tomorrow.slang_time()
'23 hours from now'

# Also: MayaDT.from_iso8601(...)
>>> tomorrow.iso8601()
'2017-02-10T22:17:01.445418Z'

# Also: MayaDT.from_rfc2822(...)
>>> tomorrow.rfc2822()
'Fri, 10 Feb 2017 22:17:01 GMT'

# Also: MayaDT.from_rfc3339(...)
>>> tomorrow.rfc3339()
'2017-02-10T22:17:01.44Z'

>>> tomorrow.datetime()
datetime.datetime(2016, 12, 16, 15, 11, 30, 263350, tzinfo=<UTC>)

# Automatically parse datetime strings and generate naive datetimes.
>>> scraped = '2016-12-16 18:23:45.423992+00:00'
>>> maya.parse(scraped).datetime(to_timezone='US/Eastern', naive=True)
datetime.datetime(2016, 12, 16, 13, 23, 45, 423992)

>>> rand_day = maya.when('2011-02-07', timezone='US/Eastern')
<MayaDT epoch=1297036800.0>

# Maya speaks Python.
>>> m = maya.MayaDT.from_datetime(datetime.utcnow())
>>> print(m)
Wed, 20 Sep 2017 17:24:32 GMT

>>> m = maya.MayaDT.from_struct(time.gmtime())
>>> print(m)
Wed, 20 Sep 2017 17:24:32 GMT

>>> m = maya.MayaDT(time.time())
>>> print(m)
Wed, 20 Sep 2017 17:24:32 GMT

>>> rand_day.day
7

>>> rand_day.add(days=10).day
17

# Always.
>>> rand_day.timezone
UTC

# Range of hours in a day:
>>> maya.intervals(start=maya.now(), end=maya.now().add(days=1), interval=60*60)
<generator object intervals at 0x105ba5820>

# snap modifiers
>>> dt = maya.when('Mon, 21 Feb 1994 21:21:42 GMT')
>>> dt.snap('@d+3h').rfc2822()
'Mon, 21 Feb 1994 03:00:00 GMT'
```



As you can see, the maya library can handle a wide array of time manipulation. This is wonderful as we're moving between the different formats of time. Using it we can also make time manipulation more robust over time. 

Simply adding the text `ten_ago = maya.now().sub(days=10)` allows us to get 10 days prior to the current point in time. We could then take that time data then convert into epoch required to query our data.

`ten_ago._epoch` - This allows us to get an `epoch` to the milisecond.

In [8]:
def add_time(item:dict, _time:float, rel_abs="absolute"):
    """ Adds time to the dictionaries we query. """
    if rel_abs == "absolute":
        item['timestamp'] = _time
    else:
        item['time'] = _time
    return item

In [9]:
def generate_dicts(data, _time, timestamp):
    relative = copy(data)
    absolute = copy(data)
    relative['time'] = _time
    absolute['timestamp'] = _time
    return {
        "relative": relative,
        "absolute": absolute
    }

In [10]:
def sorted_z_to_dict(zset:List[Set], rel_abs="absolute"):
    if len(zset) == 0 or abs_rel not in ["absolute", "relative"]:
        return []
    
    times = [x[1] for x in zset]
    dicts = [add_time(orjson.loads(x[0]), times[i], rel_abs) for i, x in enumerate(zset)]
    return dicts

In [11]:
def dictify(azset:List[Set], rzset:List[Set]):
    """Creates a single dictionary that represents the information we intend to query. """
    if len(azset) == 0 or len(rzset) == 0:
        return {}
    adict = {}
    for azs in azset:
        item, time = azs
        if item == b'{"placeholder": "place"}':
            continue
        current_item = adict.get(item, {})
        current_item['timestamp'] = time
        adict[item] = current_item
    
    # Set the relative time
    for rzs in rzset:
        item, time = rzs
        if item == b'{"placeholder": "place"}':
            continue
        current_item = adict.get(item, {})
        current_item['time'] = time
        adict[item] = current_item
    
    return adict

In [12]:
def deserialize_dicts(dictified:dict):
    _deserialized = []
    for key, value in dictified.items():
        _key = orjson.loads(key)
        _key['time'] = value.get("time", maya.now()._epoch)
        _key['timestamp'] = value.get("timestamp", maya.now()._epoch)
        _deserialized.append(_key)
    return _deserialized

In [13]:
def check_time(_time:float, _timestamp:float, local_time:float, local_timestamp:float):
    current_time = maya.now()._epoch
    
    
    if local_time is not None:
        _time = local_time
    elif _time is None:
        _time = current_time
    if local_timestamp is not None:
        _timestamp = local_timestamp
    elif _timestamp is None:
        _timestamp = current_time
    
    return {
        "time": _time,
        "timestamp": _timestamp
    }

In [14]:
def separate_time_data(data:dict, _time:float=None, _timestamp:float=None):
    local_time = data.pop("time", None)
    local_timestamp = data.pop("timestamp", None)
    timing = check_time(_time, _timestamp, local_time, local_timestamp)
    return data, timing

## Save Functions

* **save** - Save a single record at a specified time. The specified time will have both **relative** and **absolute** time. 
    - **Relative time** - This is the time specified by the data source. This is like the index inside of a timeseries dataframe. We'll query these times by epoch time.
    - **Absolute time** This is the time the record is saved into the database. The general idea here is that you'll be able to get the data in the order we entered it in as well as the actual timing of the data. We save the data in two *zlists* to represent how we want to query the data.
* **save_many**
    * Exactly the same as above, only with lots of data at once. This will be useful when we want to save lots of data representing a single data source.

In [15]:
def save(query, data, _time=None, _timestamp=None):
    if not helper.validate_query(query):
        return 
    _hash = helper.generate_hash(query)
    
    
    query.update(data)
    
    
    
    data, timing = separate_time_data(query, _time, _timestamp)
    
    relative_time_key = f"{_hash}:rlist"
    absolute_time_key = f"{_hash}:alist"
    


    
    # Generate Data
    mono = orjson.dumps(data)
    relative_data = {
        mono: timing["time"]
    }
    absolute_data = {
        mono: timing["timestamp"]
    }    

    redis.zadd(relative_time_key, relative_data)
    redis.zadd(absolute_time_key, absolute_data)

# Query Functions

Query all of the data according to our parameters. You'll see the conventional query key query up top. You'll see the `abs_rel` parameter inside of query_latest. The general idea here is that you'll be able to query from either the relative or absolute time factor. Try it out.


The query functions you'll have to work on are the following:

1. `query` - Get all of the records related to a given key.
2. `query_latest` - Get the `n` latest records according to our query parameters.
3. `query_between` - Query between two epoch times.
4. `query_before` - Get everything before an epoch time.
5. `query_after` - Get everything after epoch time.



## Useful Reference for `zrangebyscore`
https://stackoverflow.com/questions/31876179/redis-zrangebyscore-what-is-offset-and-count

In [16]:
def query(_query):
    if not helper.validate_query(_query):
        return 
    _hash = helper.generate_hash(_query)
    relative_time_key = f"{_hash}:rlist"
    absolute_time_key = f"{_hash}:alist"
    keys = redis.zrange(relative_time_key, 0, -1, withscores=True)
    akeys = redis.zrange(absolute_time_key, 0, -1, withscores=True)

    dicts = dictify(akeys, keys)
    combined = deserialize_dicts(dicts)
    return combined

In [17]:
def query_latest(_query, abs_rel="absolute"):
    if not helper.validate_query(_query) or abs_rel not in ["absolute", "relative"]:
        return 
    _hash = helper.generate_hash(_query)
    
    _current_key = ""
    if abs_rel == "absolute":
        _current_key = f"{_hash}:alist"
    else:
        _current_key = f"{_hash}:rlist"
    
    blank_keys = [(b'{"placeholder": "place"}', 0)]
    keys = redis.zrange(_current_key, -1, -1, withscores=True)
    dicts = dictify(keys, blank_keys)
    combined = deserialize_dicts(dicts)
    return combined

In [69]:
def query_latest_many(_query, abs_rel="absolute", limit:int=10):
    if not helper.validate_query(_query) or abs_rel not in ["absolute", "relative"]:
        return 
    _hash = helper.generate_hash(_query)
    
    _current_key = ""
    if abs_rel == "absolute":
        _current_key = f"{_hash}:alist"
    else:
        _current_key = f"{_hash}:rlist"
    
    blank_keys = [(b'{"placeholder": "place"}', 0)]
    keys = redis.zrange(_current_key, -limit, -1, withscores=True)
    dicts = []
    if abs_rel == "absolute":
        dicts = dictify(keys, blank_keys)
    else:
        
        dicts = dictify(blank_keys, keys)
    
    combined = deserialize_dicts(dicts)
    return combined

In [19]:
def query_between(_query, min_epoch, max_epoch, abs_rel="absolute", limit:int=10):
    if not helper.validate_query(_query) or abs_rel not in ["absolute", "relative"]:
        return 
    _hash = helper.generate_hash(_query)
    
    _current_key = ""
    if abs_rel == "absolute":
        _current_key = f"{_hash}:alist"
    else:
        _current_key = f"{_hash}:rlist"
    
    blank_keys = [(b'{"placeholder": "place"}', 0)]
    keys = redis.zrangebyscore(_current_key, min=min_epoch, max=max_epoch, withscores=True)
    dicts = []
    if abs_rel == "absolute":
        dicts = dictify(keys, blank_keys)
    else:
        
        dicts = dictify(blank_keys, keys)
    
    combined = deserialize_dicts(dicts)
    return combined

In [20]:
def query_latest_by_time(_query, max_epoch, abs_rel="absolute", limit:int=10):
    if not helper.validate_query(_query) or abs_rel not in ["absolute", "relative"]:
        return 
    _hash = helper.generate_hash(_query)
    
    _current_key = ""
    if abs_rel == "absolute":
        _current_key = f"{_hash}:alist"
    else:
        _current_key = f"{_hash}:rlist"
    
    blank_keys = [(b'{"placeholder": "place"}', 0)]
    keys = redis.zrangebyscore(_current_key, min="-inf", max=max_epoch, start=-2, num=-1, withscores=True)
    dicts = []
    if abs_rel == "absolute":
        dicts = dictify(keys, blank_keys)
    else:
        dicts = dictify(blank_keys, keys)
    
    combined = deserialize_dicts(dicts)
    return combined

In [21]:
data = web.DataReader('AAPL','yahoo',start='2018/1/1',end='2020/1/1').round(2)
data = data.astype(str)
data_json = data.to_json(orient='index')

In [22]:
# data_json

In [23]:
episode = uuid.uuid1().hex

In [24]:
save({"type": "hello", "episode": episode}, {"name":"world"}, _time=(maya.now()._epoch + 3600))
save({"type": "hello", "episode": episode}, {"name":"world", "my": "world"}, _time=(maya.now()._epoch))

In [25]:
query_latest({"type": "hello", "episode": episode}, abs_rel="relative")

[{'type': 'hello',
  'episode': 'e90a7a60409b11eabb1e80c5f21e8205',
  'name': 'world',
  'time': 1580084896.2402685,
  'timestamp': 1580088496.2286704}]

In [26]:
query_latest({"type": "hello", "episode": episode})

[{'type': 'hello',
  'episode': 'e90a7a60409b11eabb1e80c5f21e8205',
  'name': 'world',
  'my': 'world',
  'time': 1580084896.2522504,
  'timestamp': 1580084896.2301676}]

In [27]:
query_latest_many({"type": "hello", "episode": episode}, abs_rel="relative", limit=100)

[{'type': 'hello',
  'episode': 'e90a7a60409b11eabb1e80c5f21e8205',
  'name': 'world',
  'my': 'world',
  'time': 1580084896.230129,
  'timestamp': 1580084896.263171},
 {'type': 'hello',
  'episode': 'e90a7a60409b11eabb1e80c5f21e8205',
  'name': 'world',
  'time': 1580088496.2286704,
  'timestamp': 1580084896.2631752}]

In [28]:
all_items = [
    {
        "data": {"name":"world", "my": "world", "_id": uuid.uuid4().hex},
        "timestamp": maya.now()._epoch
    },
    {
        "data": {"name":"world", "my": "world", "_id": uuid.uuid4().hex},
        "timestamp": maya.now()._epoch
    },
    {
        "data": {"name":"world", "my": "world", "_id": uuid.uuid4().hex},
        "timestamp": maya.now()._epoch
    },
    {
        "data": {"name":"world", "my": "world", "_id": uuid.uuid4().hex},
        "timestamp": maya.now()._epoch
    },
    {
        "data": {"name":"world", "my": "world", "_id": uuid.uuid4().hex},
        "timestamp": maya.now()._epoch
    },
    {
        "data": {"name":"world", "my": "world", "_id": uuid.uuid4().hex},
        "timestamp": maya.now()._epoch
    },
    {
        "data": {"name":"world", "my": "world", "_id": uuid.uuid4().hex},
        "timestamp": maya.now()._epoch
    },
]

In [29]:
def convert_to_storable(items:list):
    savable = {}
    for item in items:
        item_json = orjson.dumps(item.get("data", {}))
        timestamp = float(item.get("timestamp", maya.now()._epoch))
        savable[item_json] = timestamp
    
    return savable

In [30]:
def convert_to_storable_json(json_string):
    
    savable = {}
    for key, value in orjson.loads(json_string).items():
        item_json = orjson.dumps(value)
        timestamp = float(key) * 0.001
        savable[item_json] = timestamp
    return savable

In [31]:
convert_to_storable(all_items)

{b'{"name":"world","my":"world","_id":"09fe84d212464c46ba0bdcd555654036"}': 1580084897.1668813,
 b'{"name":"world","my":"world","_id":"3de2bcfb82124f18957b15c751664085"}': 1580084897.1669273,
 b'{"name":"world","my":"world","_id":"a7363af47ecf4ad8b06d190ef4d27ade"}': 1580084897.166964,
 b'{"name":"world","my":"world","_id":"7c0a58ce2b244054aab627b527eb94be"}': 1580084897.1669943,
 b'{"name":"world","my":"world","_id":"4e9b76627d674666bb4480d6ded6fd1c"}': 1580084897.1670585,
 b'{"name":"world","my":"world","_id":"08b9d90459da4f86a33bbeb3316f26a5"}': 1580084897.1671407,
 b'{"name":"world","my":"world","_id":"07f5033affac460598c34d7a51c881a8"}': 1580084897.1671963}

In [32]:
def save_many(query, data:list, _time=None, _timestamp=None):
    if not helper.validate_query(query):
        return 
    _hash = helper.generate_hash(query)
    
#     query.update(data)
    
#     data, timing = separate_time_data(query, _time, _timestamp)
    
    relative_time_key = f"{_hash}:rlist"
    absolute_time_key = f"{_hash}:alist"

    data = convert_to_storable(all_items)
    print(data)
    redis.zadd(relative_time_key, data)
    redis.zadd(absolute_time_key, data)

### Delete Commands

1. `delete` - Get all of the records related to a given key.
2. `delete_latest` - Get the `n` latest records according to our query parameters.
3. `delete_between` - Query between two epoch times.
4. `delete_before` - Get everything before an epoch time.
5. `delete_after` - Get everything after epoch time.

In [33]:
def delete(_query):
    if not helper.validate_query(_query):
        return 
    _hash = helper.generate_hash(_query)
    relative_time_key = f"{_hash}:rlist"
    absolute_time_key = f"{_hash}:alist"
    keys = redis.zrange(relative_time_key, 0, -1, withscores=True)
    akeys = redis.zrange(absolute_time_key, 0, -1, withscores=True)
    
    # remove both akeys and keys
    values = [key[0] for key in keys]
    avalues = [key[0] for key in akeys]
    
    rrev = redis.zrem(relative_time_key, *values)
    arev = redis.zrem(absolute_time_key, *avalues)

    return rrev, arev

In [34]:
def delete_latest(_query, abs_rel="absolute"):
    if not helper.validate_query(_query) or abs_rel not in ["absolute", "relative"]:
        return 
    _hash = helper.generate_hash(_query)
    
    _current_key = ""
    if abs_rel == "absolute":
        _current_key = f"{_hash}:alist"
    else:
        _current_key = f"{_hash}:rlist"
    
    blank_keys = [(b'{"placeholder": "place"}', 0)]
    keys = redis.zrem(_current_key, -1, -1, withscores=True)
    
    values = [key[0] for key in keys]
   
    return redis.zrem(_current_key, *values)

In [35]:
def delete_latest_many(_query, abs_rel="absolute", limit:int=10):
    if not helper.validate_query(_query) or abs_rel not in ["absolute", "relative"]:
        return 
    _hash = helper.generate_hash(_query)
    
    _current_key = ""
    if abs_rel == "absolute":
        _current_key = f"{_hash}:alist"
    else:
        _current_key = f"{_hash}:rlist"
    
    blank_keys = [(b'{"placeholder": "place"}', 0)]
    keys = redis.zrange(_current_key, -limit, -1, withscores=True)
    
    values = [key[0] for key in keys]
    if len(values) == 0:
        return
    
    redis.zrem(_current_key, *values)

In [67]:
def delete_between(_query, min_epoch, max_epoch, abs_rel="absolute", limit:int=10):
    if not helper.validate_query(_query) or abs_rel not in ["absolute", "relative"]:
        return 
    _hash = helper.generate_hash(_query)
    
    _current_key = ""
    abs_key = f"{_hash}:alist"
    rel_key = f"{_hash}:rlist"
    if abs_rel == "absolute":
        _current_key = f"{_hash}:alist"
    else:
        _current_key = f"{_hash}:rlist"
    
    
    is_success = redis.zrangebyscore(_current_key, min=min_epoch, max=max_epoch)
    if len(is_success) == 0:
        print(0)
        return
    deleted1 = redis.zrem(rel_key, *is_success)
    deleted2 = redis.zrem(abs_key, *is_success)
    
    print(deleted1)
    print(deleted2)

## Head-Based Backtesting
---
Here you'll be shifting the head forward by a given amount over time. The `head` is an epoch representing where we'll be querying everything from. Combined with either the `query_before` or `query_between` function we can get information on a rolling basis.

Your task here will be to create a class that would:

1. Define a `head`
    * This should be an epoch that is the start of a backtest. We can get the beginning of a data source and get the n seconds/hours/days after.
2. Define an `interval` that we'll move the head forward.
    * For example, `1 hour` would be 3600 seconds in _epoch time.
3. Define a `step` function that would push the head forward by a given interval
4. Save the `head` into redis consistently as you update it.
5. Apply a distributed lock around the head to ensure that the head isn't overwritten by accident.
    * A `redis-py` [lock](https://github.com/andymccurdy/redis-py/blob/master/redis/lock.py) polls a given key and prevents the usage of that key while the key is getting worked on.
    

```py
with lock(redis, "key_name"):
    """Do work here"""
    pass
```

# Experiments

We're going to run through the following steps:

1. Download a dataframe worth of data
2. Save a dataframe into database (assume a time index in implicitly installed)
3. Get the latest time by time index

In [37]:
def get_current_abs_time(data:dict):
    _data = copy(data)
    for k, v in data.items():
        _data[k] = maya.now()._epoch
    
    return _data

In [52]:
def save_many(query, relative_data:list):
    if not helper.validate_query(query):
        return 
    _hash = helper.generate_hash(query)
    
    relative_time_key = f"{_hash}:rlist"
    absolute_time_key = f"{_hash}:alist"

#     relative_data = convert_dataframe_to_storable_item(data)
    absolute_data = get_current_abs_time(relative_data)
    
    redis.zadd(relative_time_key, relative_data)
    redis.zadd(absolute_time_key, absolute_data)

In [39]:
data = web.DataReader('AAPL','yahoo',start='2018/1/1',end='2020/1/1').round(2)

In [40]:
def convert_dataframe_to_storable_item(df:pd.DataFrame) -> dict:
    data = df.astype(str)
    data_json = df.to_json(orient='index')
    value = convert_to_storable_json(data_json)
    return value

In [53]:
storable = convert_dataframe_to_storable_item(data)
save_many({"type": "derp", "name": "facess"}, storable)

In [70]:
query_latest_many({"type": "derp", "name": "facess"}, abs_rel="relative", limit=100)

[{'High': 198.07,
  'Low': 194.04,
  'Open': 196.31,
  'Close': 197.0,
  'Volume': 35824800.0,
  'Adj Close': 195.67,
  'time': 1565049600.0,
  'timestamp': 1580097300.8775198},
 {'High': 199.56,
  'Low': 193.82,
  'Open': 195.41,
  'Close': 199.04,
  'Volume': 33364400.0,
  'Adj Close': 197.69,
  'time': 1565136000.0,
  'timestamp': 1580097300.8775256},
 {'High': 203.53,
  'Low': 199.39,
  'Open': 200.2,
  'Close': 203.43,
  'Volume': 27009500.0,
  'Adj Close': 202.05,
  'time': 1565222400.0,
  'timestamp': 1580097300.8775296},
 {'High': 202.76,
  'Low': 199.29,
  'Open': 201.3,
  'Close': 200.99,
  'Volume': 24619700.0,
  'Adj Close': 200.39,
  'time': 1565308800.0,
  'timestamp': 1580097300.8775332},
 {'High': 207.16,
  'Low': 203.84,
  'Open': 204.28,
  'Close': 206.5,
  'Volume': 27620400.0,
  'Adj Close': 205.88,
  'time': 1565913600.0,
  'timestamp': 1580097300.877539},
 {'High': 212.73,
  'Low': 210.03,
  'Open': 210.62,
  'Close': 210.35,
  'Volume': 24413600.0,
  'Adj Close':

In [71]:
def query_latest_by_time(_query, max_epoch, abs_rel="absolute", limit:int=10):
    if not helper.validate_query(_query) or abs_rel not in ["absolute", "relative"]:
        return 
    _hash = helper.generate_hash(_query)
    
    _current_key = ""
    if abs_rel == "absolute":
        _current_key = f"{_hash}:alist"
    else:
        _current_key = f"{_hash}:rlist"
    
    blank_keys = [(b'{"placeholder": "place"}', 0)]
    keys = redis.zrangebyscore(_current_key, min=max_epoch, max="+inf", start=0, num=1, withscores=True)
    dicts = []
    if abs_rel == "absolute":
        dicts = dictify(keys, blank_keys)
    else:
        dicts = dictify(blank_keys, keys)
    
    combined = deserialize_dicts(dicts)
    if len(combined) == 0:
        return {}
    return combined[0]

In [72]:
# _query, max_epoch, abs_rel="absolute", limit:int=10
query_latest_by_time({"type": "derp", "name": "facess"}, max_epoch=1576627200.0, abs_rel="relative")

{'High': 281.9,
 'Low': 279.12,
 'Open': 279.8,
 'Close': 279.74,
 'Volume': 29007100.0,
 'Adj Close': 279.74,
 'time': 1576627200.0,
 'timestamp': 1580106170.4427986}

In [48]:
# 1580002907.3769891
1565827200.0 - 1565568000.0

259200.0

In [68]:
delete_between({"type": "derp", "name": "facess"}, 1565568000.0, 1565827200.0, abs_rel="relative")

0
